In [1]:
#r ".\binaries\BoSSSpad.dll"
#r ".\binaries\XESF.dll"
#r ".\binaries\ApplicationWithIDT.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Platform.LinAlg;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using ApplicationWithIDT;
using XESF;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();


In [2]:
using System.IO;
static void AppendRowToCSV(string filePath, double[] rowData)
    {
        // Create a StreamWriter in append mode to append rows to the CSV file
        using (StreamWriter writer = new StreamWriter(filePath, true))
        {
            int cols = rowData.Length;

            // Write double values of the new row to the file
            for (int i = 0; i < cols; i++)
            {
                writer.Write(rowData[i]);

                // Add a comma after each value except for the last one in a row
                if (i < cols - 1)
                {
                    writer.Write(";");
                }
            }

            // Move to the next line after writing the row
            writer.WriteLine();
        }
    }
    static void WriteCSV(string filePath, string[] data)
    {
        // Create a StreamWriter to write to the CSV file
        using (StreamWriter writer = new StreamWriter(filePath))
        {
            int cols = data.Length;

            // Loop through the data array and write to the file
                for (int j = 0; j < cols; j++)
                {
                    // Write each cell value to the file
                    writer.Write(data[ j]);

                    // Add a comma after each cell value except for the last one in a row
                    if (j < cols - 1)
                    {
                        writer.Write(";");
                    }
                }

                // Move to the next line after each row
                writer.WriteLine();
            
        }
    }
    static void DeleteCSVFiles()
    {
        try
        {
            // Get all CSV files in the specified directory
            string[] csvFiles = Directory.GetFiles("*.csv");

            // Delete each CSV file
            foreach (string file in csvFiles)
            {
                File.Delete(file);
                Console.WriteLine($"Deleted: {file}");
            }
        }
        catch (Exception ex)
        {
            Console.WriteLine($"Error deleting CSV files: {ex.Message}");
        }
    }

## Measure wall-clock time for computation of sensitivities and residuals 

In [3]:
using System.Diagnostics;
BoSSS.Solution.Application.InitMPI();
//DeleteCSVFiles();

int nRefs=3;
var optLsTypes=new OptiLevelSetType[] {OptiLevelSetType.SinglePhaseField,OptiLevelSetType.SplineLevelSet,OptiLevelSetType.SpecFemField};
var CellsVsRuntime_dRdPhi = MultidimensionalArray.Create(nRefs,optLsTypes.Length +1);
var CellsVsRuntime_Residuals = MultidimensionalArray.Create(nRefs,optLsTypes.Length +1);
var CellsVsRuntime_dRdU = MultidimensionalArray.Create(nRefs,optLsTypes.Length +1);
string[] stringArray = new string[] { "nCells" }.Concat(optLsTypes.Select(x => x.ToString())).ToArray();

//Create files for results
string filePathdRdPhi = "TimeDRdPhi.csv";
string filePathdRdU = "TimedRdU.csv";
string filePathResiduals = "TimeResiduals.csv";
string filePathDOFs = "DOFs.csv";
WriteCSV(filePathdRdPhi,stringArray);
WriteCSV(filePathdRdU,stringArray);
WriteCSV(filePathResiduals,stringArray);
WriteCSV(filePathDOFs,stringArray);

for(int n =1; n< nRefs;n++){
    //counter for level set type
    int iLS=1;
    //save cell resolution
    int reso= (int) Math.Pow(2,n);
    CellsVsRuntime_dRdPhi[n,0]=reso*reso;
    double[] dRdPhiResults= new double[optLsTypes.Length +1];
    dRdPhiResults[0]=reso*reso;
    double[] dRdUResults=dRdPhiResults.CloneAs();
    double[] resiudalResults=dRdPhiResults.CloneAs();
    double[] DOFs=dRdPhiResults.CloneAs();
    foreach(OptiLevelSetType lstype in optLsTypes){
        var pXESF = new XESFMain();
        var C = XESFHardCodedControl.XDGWedgeFlow_TwoLs_Base(
            dbPath:null,
            optiLSDegree: 3,
            lsDegree: 1,
            shocksetup: ApplicationWithIDT.GetLevelSet.FromFunction,
            optiLevelSetType: lstype,
            initialValue: ApplicationWithIDT.GetInitialValue.FromFunctionPerSpecies,
            MaxIterations: 1,
            dgDegree: 3,
            numOfCellsX: reso,
            numOfCellsY: reso,
            initialAngle_shockLS: 32,
            PlotInterval: -1,
            interfaceFluxLS2: XESF.Fluxes.ConvectiveInterfaceFluxes.GodunovInterface,
            bulkFlux: XESF.Fluxes.ConvectiveBulkFluxes.OptimizedHLLC,
            agg: 0.4
        );


        pXESF.Init(C);    
        //pXESF.RunSolverMode();
        pXESF.InitializeEverything();
        

        //Measure time for creation of Jacobians dRdPhi
        Stopwatch stopwatch = new Stopwatch();stopwatch.Start();
        pXESF.FD_LevelSet();
        stopwatch.Stop();
        CellsVsRuntime_dRdPhi[n,iLS] =stopwatch.ElapsedMilliseconds;
        dRdPhiResults[iLS]=stopwatch.ElapsedMilliseconds;

        //Measure time for creation of Jacobians dRdU
        stopwatch = new Stopwatch();stopwatch.Start();
        pXESF.Oproblem.GetJacobians(pXESF.ConservativeFields, pXESF.XSpatialOperator.LinearizationHint);
        stopwatch.Stop(); 
        CellsVsRuntime_dRdU[n,iLS] =stopwatch.ElapsedMilliseconds;
        dRdUResults[iLS]=stopwatch.ElapsedMilliseconds;

        //Measure time for evaluation of the Residuals
        stopwatch = new Stopwatch();stopwatch.Start();
        pXESF.ComputeResiduals();
        stopwatch.Stop(); Console.WriteLine("Elapsed Time: " + stopwatch.ElapsedMilliseconds + "ms");
        CellsVsRuntime_Residuals[n,iLS] =stopwatch.ElapsedMilliseconds;
        resiudalResults[iLS]=stopwatch.ElapsedMilliseconds;

        DOFs[iLS]=pXESF.LevelSetOpti.GetLength();

        iLS++;
    }
    // Append the new row to the existing CSV file
    AppendRowToCSV(filePathdRdPhi, dRdPhiResults);
    AppendRowToCSV(filePathdRdU, dRdUResults);
    AppendRowToCSV(filePathResiduals, resiudalResults);
    AppendRowToCSV(filePathDOFs, DOFs);
}

Affinity reported from Win32 API: 0, 1, 2, 3, 4, 5, 6, 7;
Grid Edge Tags changed.
Session ID: 00000000-0000-0000-0000-000000000000, DB path: 'EMPTY'
Session directory 'EMPTY\sessions\00000000-0000-0000-0000-000000000000'.
Grid repartitioning method: METIS
Grid repartitioning options: 
Number of cell Weights: 0


Error: System.ArgumentException: length of each dimension in species table must be 2, lower bound must be 0
   at BoSSS.Foundation.XDG.LevelSetTracker.CollectSpeciesNamesRecursive(Int32 recdeph, Int32[] indices) in C:\experimental\public\src\L2-foundation\BoSSS.Foundation.XDG\LevelSetTracker.cs:line 369
   at BoSSS.Foundation.XDG.LevelSetTracker.ConstructorCommon(GridData griData, Int32 __NearRegionWidth, Array SpeciesTable, MomentFittingVariants cutCellQuadratureType, ILevelSet[] levSets) in C:\experimental\public\src\L2-foundation\BoSSS.Foundation.XDG\LevelSetTracker.cs:line 290
   at BoSSS.Foundation.XDG.LevelSetTracker.ConstructorCommon(GridData BackgroundGrid, MomentFittingVariants cutCellquadType, Int32 __NearRegionWidth, Array SpeciesTable, ILevelSet[] levSets) in C:\experimental\public\src\L2-foundation\BoSSS.Foundation.XDG\LevelSetTracker.cs:line 152
   at BoSSS.Foundation.XDG.LevelSetTracker..ctor(GridData BackgroundGrid, MomentFittingVariants cutCellquadType, Int32 __Nea

In [4]:
public PlotFormat GetFormat(int count){
    
    var allPT = new BoSSS.Solution.Gnuplot.PointTypes[] { PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle,PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle};
    var allC = new BoSSS.Solution.Gnuplot.LineColors[] { LineColors.Blue, LineColors.Black, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red};
    var Fmt = new PlotFormat();
    Fmt.PointSize = 0.8;
    Fmt.LineWidth = 1;    
    Fmt.Style     = Styles.LinesPoints;

    Fmt.LineColor = allC[count];
    Fmt.PointType =  allPT[count];
    return Fmt;
}

In [5]:
var plot = new Plot2Ddata();
var cells = CellsVsRuntime_dRdPhi.Co;
int count=0;
foreach(OptiLevelSetType lstype in optLsTypes){
    runtimes.
    plot.AddDataGroup(lstype.ToString,cells,runtimes, GetFormat(count));
    count++;
}
plot.Xlabel="number of Cells"";
plot.LogY = true;
plot.LogX = true;
plot.ShowXtics = true;
plot.ToGnuplot().PlotSVG(xRes:1200,yRes:600)

Error: (9,30): error CS1002: ; expected
(9,30): error CS1010: Newline in constant
(9,32): error CS1002: ; expected